In [1]:
from bb_behavior import plot
from bb_behavior import db as bbdb
import bb_behavior
import pandas as pd
import os
import sys
sys.path.append('/home/mi/rrszynka/mnt/janek/Beesbook-janek/Python-modules/') #For bee_helpers, file_helpers and cache
from bee_cache import Cache, CacheType, CacheFormat; c = Cache()
import datetime
from bb_behavior import db as bbdb
from skimage.morphology import rectangle, closing
import datetime
import numpy as np
from tqdm import tqdm
import random
#%%

/usr/local/lib/python3.5/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [17]:
INTERVAL_SIZE = 30
FPS = 3

# Get the pre-generated list of 50 random exits  
random_exits = c.load('50_random_exits')

# Get all cached days (or consider excluding some)
presences = c.load_all_presence_caches()

presences_bin = []
for (date, presence) in tqdm(presences):
    presence_bin = binarize_presence(presence)
    presences_bin.append((date, presence_bin))
#%%

  0%|          | 0/62 [00:00<?, ?it/s]

Collected 62/62 presence caches (all that are currently downloaded).


100%|██████████| 62/62 [00:10<00:00, 21.27it/s]


In [31]:
# GENERATE VIDEOS
for i, row in random_exits.iterrows():
    generate_presence_video(row.bee_id, row.timestamp, row.frame)

2016-07-20 01:41:00 202


KeyboardInterrupt: 

In [27]:
#%%
def generate_presence_video(bee, timestamp, frame):
    # ASK: can this be used given hiccups? or just take days w/o hiccups?
    interval_margin = 2
    frames_per_interval = INTERVAL_SIZE*FPS
    frame_margin = interval_margin*frames_per_interval #30s * 3FPS

    name = str(bee)+'_random_exit'
    path = '/home/mi/rrszynka/mnt/janek/caches/Videos/' + name + ".mp4"
    if os.path.isfile(path):
        print('breaking out')
        return
    video = plot.plot_bees(bee_ids=[bee],
                       frame_id=frame, frame_margin=frame_margin,
                       path_alpha=None, bt_export=None, # '/home/mi/rrszynka/mnt/janek/caches/Videos/bt_export/'+name+".json",
                       plot_labels = True, plot_markers=True)

    intervals = len(video._frames)/(interval_margin*2)

    #TODO: check this conversion from timestamp to interval against the conversion that goes the other way
    middle_timestamp = timestamp

    secs = time_to_secs(middle_timestamp)
    interval_index_mid = int(secs/30)
    print(timestamp, interval_index_mid)
    start_interval = interval_index_mid - interval_margin

    _, presence_df = next(((d, p) for d , p in presences if d == timestamp.date()), None)
    _, bin_presence_df = next(((d, p) for d , p in presences_bin if d == timestamp.date()), None)


    pres_row = presence_df.loc[bee]
    bin_pres_row = bin_presence_df.loc[bee]

    for i, frame in enumerate(video._frames):
        interval_index = (i // frames_per_interval)
        #PRINT: pres_score, pres_bin, cam_id, bee_id (or save in file), exit_dist(would be nice)
        frame._title = "\n invl: " + str(interval_index) + "/" + str(interval_margin*2 - 1) + ",  P_S: " + str(pres_row[start_interval + interval_index]) + ",  PB: " + str(bin_pres_row[start_interval + interval_index])
    video._crop_margin = None
    video.get_video(save_to_path='/home/mi/rrszynka/mnt/janek/caches/Videos/' + name + ".mp4")

In [ ]:
# plot_presence(presx)
# plot_presence(presbin)

In [3]:
def time_to_secs(time):
    return time.hour * 3600 + time.minute * 60 + time.second

    
def binarize_presence(presence_for_day_df):
    # TODO: running this on an already-binarized series will return all zeros,
    # (and that's not what we want) - changing the threshold solves it, but try sth else

    ys = presence_for_day_df.copy()
    index = ys.index
    if ys[ys>1].sum().sum() == 0:
        #Consider this already binarized, make no changes
        return ys

    ys[ys>90] = 90
    ys[ys>45] = 90
    ys[ys<=45] = 0 #TODO: consult: what should the threshold be
    ys[ys==90] = 1
    ys = closing(ys, rectangle(1,15))
    ys = pd.DataFrame(ys, index=index)
    ys.index.name = 'bee_id'
    return ys